In [2]:
import shapefile
import shapely
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import pandas as pd

In [3]:
sf = shapefile.Reader("./VIC_LOCALITY_POLYGON_shp")  #all the shape related files are read
recs = sf.records()   #Extracting the records from the shape files

suburbs = []  #Empty list to store the suburb names
#Iterating through the records
for i in recs:
    suburbs.append(i[6])   #appending the suburb name
    
#Extracting the polygon data
feature = sf.shapeRecords()
boundaries = []  #Empty list to store the polygon coordintes
#Iterating through each shape 
for shape in feature:
    polygon_dict = shape.shape.__geo_interface__    #getting the dictionary for each region
    polygon = []   #Empty list to store the each coordinate of individual polygon
    #Iterating through the polygons present in the shape file
    for point in polygon_dict['coordinates']:
        #Iterating through each coordinate of each polygon
        for cor in point:
            polygon.append(cor)   #Appending the coordinates
    boundaries.append(polygon)   #Appending the boundaries of each polygon

In [8]:
crashes = pd.read_csv('bikecrashesfinal.csv')

In [9]:
crashes.head()

,X,Y,OBJECTID,ACCIDENT_NO,ABS_CODE,ACCIDENT_STATUS,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,...,DEG_URBAN_ALL,LGA_NAME_ALL,REGION_NAME_ALL,SRNS,SRNS_ALL,RMA,RMA_ALL,DIVIDED,DIVIDED_ALL,STAT_DIV_NAME
0,144.973222,-37.807724,3402746,T20130015892,ABS to receive accident,Finished,1/8/2013,18.14.00,Yes,Collision with vehicle,...,MELB_URBAN,"YARRA,MELBOURNE",METROPOLITAN NORTH WEST REGION,NaN,NaN,Arterial Other,Arterial Other,Divided,"Div,Undiv",Metro
1,144.956740,-37.754570,3402754,T20130015903,ABS to receive accident,Finished,1/8/2013,17.38.00,No,Collision with vehicle,...,MELB_URBAN,MORELAND,METROPOLITAN NORTH WEST REGION,NaN,NaN,Arterial Other,"Arterial Other,Local Road",Undivided,Undiv,Metro
2,144.973320,-37.795216,3402760,T20130015919,ABS to receive accident,Finished,2/8/2013,09.55.00,No,Collision with vehicle,...,MELB_URBAN,MELBOURNE,METROPOLITAN NORTH WEST REGION,NaN,NaN,Local Road,Local Road,Divided,Div,Metro
3,145.086100,-37.924472,3402768,T20130015938,ABS to receive accident,Finished,1/8/2013,18.39.00,Yes,Collision with vehicle,...,MELB_URBAN,"MONASH,KINGSTON",METROPOLITAN SOUTH EAST REGION,NaN,NaN,Arterial Other,Arterial Other,Undivided,Undiv,Metro
4,144.992865,-37.854519,3402771,T20130015945,ABS to receive accident,Finished,1/8/2013,08.00.00,No,Collision with vehicle,...,MELB_URBAN,STONNINGTON,METROPOLITAN SOUTH EAST REGION,NaN,NaN,Local Road,Local Road,Undivided,Undiv,Metro


In [10]:
len(crashes)

6706

In [11]:
#Fnction to get the suburb name given a latitude and longitude
def in_polygon(longitude, latitude):
    point = Point(longitude, latitude)  #Creating a pont using the lat and long
    i = 0   #Initial value of i
    #Looping through all the polygons
    while i <len(boundaries):
        #Pulling the coordinates of a polygon
        polygon = Polygon(boundaries[i])
        #Checking of the point is in polygon
        if polygon.contains(point):
            return i  #retuurning the index
        i += 1
                 
crashes_suburbs = []   #Empty list to store the suburb names
#Looping through all the rows on data frame
for j in range(len(crashes)):
    #i is generated using the point in polygon function             
    suburb_index = in_polygon(float(crashes['LONGITUDE'][j]),float(crashes['LATITUDE'][j]))
    #Appending the Suburb name 
    crashes_suburbs.append(suburbs[suburb_index])

print(len(crashes_suburbs))  #printing the length of suburbs list

6706


In [12]:
crashes['suburb'] = crashes_suburbs #Adding the suburb column

In [15]:
crashes.tail()

,X,Y,OBJECTID,ACCIDENT_NO,ABS_CODE,ACCIDENT_STATUS,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,...,LGA_NAME_ALL,REGION_NAME_ALL,SRNS,SRNS_ALL,RMA,RMA_ALL,DIVIDED,DIVIDED_ALL,STAT_DIV_NAME,suburb
6701,144.988035,-37.810915,3473680,T20180019203,ABS to receive accident,Finished,10/10/2018,18.00.00,Yes,No collision and no object struck,...,MELBOURNE,METROPOLITAN NORTH WEST REGION,NaN,NaN,Local Road,Local Road,Divided,Div,Metro,EAST MELBOURNE
6702,144.943980,-37.800207,3473692,T20180019676,ABS to receive accident,Finished,20/10/2018,12.20.00,No,Collision with vehicle,...,MELBOURNE,METROPOLITAN NORTH WEST REGION,NaN,NaN,Arterial Other,Arterial Other,Divided,Div,Metro,NORTH MELBOURNE
6703,144.964817,-37.811747,3473707,T20180018741,ABS to receive accident,Finished,11/9/2018,07.40.00,No,Collision with vehicle,...,MELBOURNE,METROPOLITAN NORTH WEST REGION,NaN,NaN,Local Road,Local Road,Undivided,Undiv,Metro,MELBOURNE
6704,145.327595,-37.815175,3473719,T20180018768,ABS to receive accident,Finished,6/10/2018,08.08.00,No,Collision with vehicle,...,YARRA RANGES,METROPOLITAN SOUTH EAST REGION,NaN,NaN,Arterial Other,Arterial Other,Divided,Div,Metro,MONTROSE
6705,144.984016,-37.887111,3473727,T20180019217,ABS to receive accident,Finished,13/10/2018,12.03.00,No,Collision with vehicle,...,PORT PHILLIP,METROPOLITAN NORTH WEST REGION,NaN,NaN,Arterial Other,Arterial Other,Undivided,Undiv,Metro,ELWOOD


In [17]:
crashes
#Writing data to a Csv file
crashes.to_csv('bikecrasheswithsuburbsfinal2.csv',sep='\t',encoding='utf-32', index=False)